# Lane Detection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [ ]:
def canny_edge_detector(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny  

In [ ]:
def ROI_mask(img):
    h = img.shape[0]
    w = img.shape[1] 
    poly = np.array([[(0, h), (round(w/2), round(h/2)), (1000, h)]]) 
    
    mask = np.zeros_like(img) 
    cv2.fillPoly(mask, poly, 255) 
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

In [ ]:
def get_coordinates(img, params):  
    m, y_int = params 
    y1 = img.shape[0]     
    y2 = int(y1 * (3/5))
    x1 = int((y1 - y_int) / m)
    x2 = int((y2 - y_int) / m) 
    
    return np.array([x1, y1, x2, y2])

In [ ]:
def avg_lines(img, lines): 
    
    left = [] 
    right = [] 
    
    for l in lines: 
        x1, y1, x2, y2 = l.reshape(4)
        params = np.polyfit((x1, x2), (y1, y2), 1)  
        m = params[0] 
        y_int = params[1] 
        
        if m < 0: 
            left.append((m, y_int)) # Negative slope = left lane
        else: 
            right.append((m, y_int)) # Positive slope = right lane
        
    left_avg = np.average(left, axis=0) 
    right_avg = np.average(right, axis=0) 
    left_line = get_coordinates(img, left_avg) 
    right_line = get_coordinates(img, right_avg)
    
    return np.array([left_line, right_line])

In [ ]:
def combine_imgs(img, lines, thickness): 
    print(lines)
    
    line_img = np.zeros_like(img)
    
    if lines is not None: 
        for x1, y1, x2, y2 in lines:
            cv2.line(line_img, (x1, y1), (x2, y2),  [0, 0, 255] , thickness)

    combined_img = cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0)
    
    return combined_img


### Load Image

In [ ]:
img = cv2.imread("frame.png")
plt.figure()
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

### Edge Detection

In [ ]:
canny_edges_img = canny_edge_detector(img)
plt.figure()
plt.imshow(cv2.cvtColor(canny_edges_img, cv2.COLOR_BGR2RGB))
plt.show()

### Region of Interest 

In [ ]:
roi_img = ROI_mask(canny_edges_img)
plt.figure()
plt.imshow(cv2.cvtColor(roi_img, cv2.COLOR_BGR2RGB))
plt.show()

### Prediction

In [ ]:
lines = cv2.HoughLinesP(
    roi_img,
    rho=2,
    theta=np.pi/180,
    threshold=100,
    lines=np.array([]),
    minLineLength=40,
    maxLineGap=25 
)

line_img = avg_lines(img, lines) 
combined_img = combine_imgs(img, line_img, 5)
plt.figure()
plt.imshow(cv2.cvtColor(combined_img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
'''
Inspiration
1. https://github.com/d-misra/Lane-detection-opencv-python
'''